In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_dataloader, get_pos_tags, get_model_items, ModelItems, run_model_on_data_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import torch_cache
from analysis_tools.plotting import print_top_probs


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings[2][1]

used_poses = ["noun", "verb"]

/usr/local/stow/python-3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(x, template, word_ids):
    batch_size = len(word_ids)
    x_batch = x.unsqueeze(0).expand(batch_size, *([-1] * x.dim()))
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    raw_y_batch = [[]] * batch_size
    return x_batch, y_batch, y_len_batch, raw_y_batch


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)

    batches = (construct_batch_by_filling_template(x, template, word_ids[i : i + batch_size])
               for i in range(0, len(word_ids), batch_size))
    losses = []
    for batch in run_model_on_data_batches(model, batches, use_tqdm=False):
        outputs, loss = batch[4:]
        loss = loss.sum(-1)
        losses.append(loss.detach())
    losses = torch.cat(losses, 0)
    return losses


def find_index(lst, e):
    for i, x in enumerate(lst):
        if x == e:
            return i
    return None


def get_prob_dist(model, example, template, word_ids, word_cats, batch_size=256, top_k=None, words=None):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
        batch_size: the batch size used to run the model
        top_k: print top k predictions along with the ground-truth label; None if unwanted
        words: words correspond to words_ids; used in printing
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(-losses, 0)
    if top_k is not None:
        used_word_ids = word_ids
        used_words = words
        used_probs = probs
        gt_word_id = template.seq[template.idx]
        gt_word_idx = find_index(used_word_ids, gt_word_id)
        if gt_word_idx is None:  # the ground-truth word is not in the vocab of prediction
            print('ground-truth word not in the vocab of prediction')
            # append the ground-truth word to the vocab
            gt_word_idx = len(word_ids)
            used_word_ids = np.append(used_word_ids, gt_word_idx)
            used_words = np.append(used_words, idx2word[gt_word_id])
            used_probs = torch.cat([used_probs, torch.tensor([0.], dtype=probs.dtype, device=probs.device)])
        print_top_probs(used_probs, used_words, labels=torch.tensor(gt_word_idx))
    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

checkpoint_path = "checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt"
print(f"load model from {checkpoint_path}")
model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
model.to(device)

# get the pos tags of all words in vocab
# by the index of model_items produced by get_model_items
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
train_model_items = torch_cache(checkpoint_path + f'.train.cache.pt')(get_model_items)(
    model, train_dataloader_fn(), train_pos_tags, ignore_all_token_items=True)
idx2pos = np.full(vocab_size, '.', dtype='O')
for token_id, pos in train_model_items.token_items.index:
    idx2pos[token_id] = pos
idx2pos2 = np.vectorize(pos_mapping.get)(idx2pos)
for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')
word_ids = np.nonzero(np.isin(idx2pos2, used_poses))[0]
words = idx2word[word_ids]
word_cats = idx2pos2[word_ids]

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 2350
load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt
load cached pos tags: dataset_cache/saycam/t

In [5]:
splits = ['val']

for split in splits:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds, corrects = [], [], []
    for example, template in joined_templates_from_examples(examples, pos_tags):
        print(template)
        gold = pos_mapping[template.pos[template.idx]]
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, top_k=5, words=words)
        print(prob_dist)
        argmax_idx = prob_dist.argmax()
        pred = prob_dist.index[argmax_idx]
        correct = pred == gold
        print(f'{gold = }, {pred = }, {correct = }')
        golds.append(gold)
        preds.append(pred)
        corrects.append(correct)
    print(f'label distribution: ')
    frac_format = lambda a, b, prec=2: f'{a} / {b} = {a/b:.{prec}%}'
    for pos in used_poses:
        cnt = sum(gold == pos for gold in golds)
        print(f'{pos}: {frac_format(cnt, len(golds))}')
    print(f'accuracy: {frac_format(sum(corrects), len(corrects))}')

load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
(0.038) turn     | (0.916) put      (0.038) turn     (0.011) do       (0.005) go       (0.003) leave   
noun    0.003873
verb    0.996127
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we should turn on some ______ , huh ?
(0.171) lights   | (0.171) lights   (0.133) light    (0.081) markers  (0.042) bread    (0.035) toys    
noun    0.971903
verb    0.028097
dtype: float32
gold = 'noun', pred = 'noun', correct = True
here , ___ me show you .
(0.991) let      | (0.991) let      (0.002) watch    (0.002) <unk>    (0.000) try      (0.000) see     
noun    0.002316
verb    0.997684
dtype: float32
gold = 'verb', pred = 'verb', correct = True
here , let me ____ you .
(0.198) show     | (0.474) help     (0.198) show     (0.058) carry    (0.040) put      (0.028) get     
noun    0.012632
verb    0.987368
dtype: float32
gold = 'verb', pred = 'verb', correct = True
yeah it __ this this one 

(0.145) pick     | (0.643) put      (0.160) check    (0.145) pick     (0.016) take     (0.006) brought 
noun    0.00155
verb    0.99845
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we go , now can you help me , did you want to help me pick out some _______ look we have new
(0.001) clothes  | (0.221) n        (0.163) words    (0.143) eggs     (0.114) apple    (0.091) shirts  
noun    0.94079
verb    0.05921
dtype: float32
gold = 'noun', pred = 'noun', correct = True
there we go , now can you help me , did you want to help me pick out some clothes ____ we have new
(0.000) look     | (0.230) do       (0.222) books    (0.115) shirts   (0.054) typing   (0.050) helping 
noun    0.500883
verb    0.499117
dtype: float32
gold = 'verb', pred = 'noun', correct = False
there we go , now can you help me , did you want to help me pick out some clothes look we ____ new
(0.908) have     | (0.908) have     (0.048) are      (0.017) get      (0.003) make     (0.003) 've     
noun    

(0.000) strawberry | (0.127) window   (0.052) words    (0.040) bin      (0.038) ball     (0.036) sandpit 
noun    0.96443
verb    0.03557
dtype: float32
gold = 'noun', pred = 'noun', correct = True
so can you .. how ____ this .. can you put bear on top of the blanket ?
(0.800) bout     | (0.800) bout     (0.043) s        (0.033) 's       (0.020) does     (0.011) be      
noun    0.024844
verb    0.975156
dtype: float32
gold = 'verb', pred = 'verb', correct = True
so can you .. how bout this .. can you ___ bear on top of the blanket ?
(0.924) put      | (0.924) put      (0.036) get      (0.022) bring    (0.006) turn     (0.003) find    
noun    0.001507
verb    0.998493
dtype: float32
gold = 'verb', pred = 'verb', correct = True
so can you .. how bout this .. can you put ____ on top of the blanket ?
(0.087) bear     | (0.200) everything (0.187) sand     (0.087) bear     (0.076) lamb     (0.062) things  
noun    0.94714
verb    0.05286
dtype: float32
gold = 'noun', pred = 'noun', correct

(0.923) remember | (0.923) remember (0.062) know     (0.008) want     (0.003) remeber  (0.002) see     
noun    0.00021
verb    0.99979
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" do you remember how to ____ it on ?
(0.887) turn     | (0.887) turn     (0.078) put      (0.011) make     (0.004) keep     (0.003) <unk>   
noun    0.004341
verb    0.995659
dtype: float32
gold = 'verb', pred = 'verb', correct = True
but , sometimes i ______ and slip and sprawl . "
(0.996) wobble   | (0.996) wobble   (0.001) <unk>    (0.001) messed   (0.000) stumble  (0.000) twist   
noun    0.001518
verb    0.998483
dtype: float32
gold = 'verb', pred = 'verb', correct = True
but , sometimes i wobble and ____ and sprawl . "
(0.352) slip     | (0.352) slip     (0.155) nectarines (0.068) <unk>    (0.042) knife    (0.037) poos    
noun    0.891819
verb    0.108181
dtype: float32
gold = 'verb', pred = 'noun', correct = False
but , sometimes i wobble and slip and ______ . "
(0.794) sprawl   | (0.

(0.019) babies   | (0.072) kitties  (0.053) pieces   (0.037) smoke    (0.035) cousins  (0.033) blocks  
noun    0.96899
verb    0.03101
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you ____ that ball .
(0.007) love     | (0.158) threw    (0.156) got      (0.148) want     (0.110) dropped  (0.082) get     
noun    0.035776
verb    0.964224
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you love that ____ .
(0.140) ball     | (0.201) one      (0.146) book     (0.140) ball     (0.074) <unk>    (0.040) picture 
noun    0.970677
verb    0.029323
dtype: float32
gold = 'noun', pred = 'noun', correct = True
" you actually ____ to get in the stroller ?
(0.803) want     | (0.803) want     (0.045) get      (0.029) need     (0.014) going    (0.012) try     
noun    0.02742
verb    0.97258
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" you actually want to ___ in the stroller ?
(0.030) get      | (0.657) sit      (0.110) go       (0.056) come     (0.030) ge

...

(0.906) do       | (0.906) do       (0.030) have     (0.020) did      (0.019) guess    (0.013) said    
noun    0.000226
verb    0.999774
dtype: float32
gold = 'verb', pred = 'verb', correct = True
so the fence is over , but i do n't ___ where we can feed the ducks .
(0.141) see      | (0.742) know     (0.141) see      (0.048) think    (0.010) remember (0.007) mess    
noun    0.021472
verb    0.978528
dtype: float32
gold = 'verb', pred = 'verb', correct = True
so the fence is over , but i do n't see where we can ____ the ducks .
(0.231) feed     | (0.249) do       (0.247) see      (0.231) feed     (0.044) read     (0.043) hear    
noun    0.009939
verb    0.990061
dtype: float32
gold = 'verb', pred = 'verb', correct = True
so the fence is over , but i do n't see where we can feed the _____ .
(0.188) ducks    | (0.188) ducks    (0.087) tractor  (0.078) chicken  (0.067) leaves   (0.044) cars    
noun    0.974717
verb    0.025283
dtype: float32
gold = 'noun', pred = 'noun', correct = Tru

(0.081) are      | (0.538) did      (0.167) were     (0.101) is       (0.081) are      (0.049) do      
noun    0.002496
verb    0.997504
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i 'll ___ the hashbrowns , they 're in here
(0.099) get      | (0.376) do       (0.099) get      (0.050) put      (0.048) turn     (0.047) leave   
noun    0.038696
verb    0.961304
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i 'll get the __________ , they 're in here
(0.000) hashbrowns | (0.071) blocks   (0.068) water    (0.059) keys     (0.047) nectorine (0.043) pens    
noun    0.962577
verb    0.037423
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i 'll get the hashbrowns , they ___ in here
(0.382) 're      | (0.457) are      (0.382) 're      (0.085) come     (0.026) go       (0.006) cut     
noun    0.007141
verb    0.992859
dtype: float32
gold = 'verb', pred = 'verb', correct = True
here __ a boy with a birdie and there is a boy with a tower .
(0.497) is 

(0.833) are      | (0.833) are      (0.070) go       (0.022) is       (0.014) went     (0.014) pants   
noun    0.027347
verb    0.972653
dtype: float32
gold = 'verb', pred = 'verb', correct = True
___ , he is gone .
(0.003) ohp      | (0.646) see      (0.169) look     (0.016) remember (0.015) <unk>    (0.015) oops    
noun    0.136216
verb    0.863785
dtype: float32
gold = 'noun', pred = 'verb', correct = False
ohp , he __ gone .
(0.258) is       | (0.502) 's       (0.258) is       (0.133) was      (0.048) s        (0.032) gets    
noun    0.002566
verb    0.997434
dtype: float32
gold = 'verb', pred = 'verb', correct = True
ohp , he is ____ .
(0.036) gone     | (0.513) twisting (0.092) eating   (0.061) <unk>    (0.036) gone     (0.023) smiling 
noun    0.164438
verb    0.835562
dtype: float32
gold = 'verb', pred = 'verb', correct = True
____ some grapefruit ?
(0.857) want     | (0.857) want     (0.043) try      (0.020) eat      (0.012) need     (0.010) <unk>   
noun    0.011675
verb  

(0.233) look     | (0.293) see      (0.233) look     (0.118) <unk>    (0.072) let      (0.023) watch   
noun    0.268536
verb    0.731464
dtype: float32
gold = 'verb', pred = 'verb', correct = True
look , ___ , let 's help .
(0.155) see      | (0.316) <unk>    (0.240) look     (0.155) see      (0.047) sam      (0.020) wait    
noun    0.508532
verb    0.491468
dtype: float32
gold = 'verb', pred = 'noun', correct = False
look , see , ___ 's help .
(0.947) let      | (0.947) let      (0.014) mommy    (0.008) <unk>    (0.008) sam      (0.002) kitty   
noun    0.048169
verb    0.951831
dtype: float32
gold = 'verb', pred = 'verb', correct = True
look , see , let 's ____ .
(0.000) help     | (0.935) see      (0.025) go       (0.009) try      (0.004) play     (0.003) look    
noun    0.010717
verb    0.989283
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you wanna ___ it ?
(0.011) see      | (0.531) do       (0.131) try      (0.065) pat      (0.022) hold     (0.020) eat     
nou

(0.074) hippo    | (0.752) sam      (0.074) hippo    (0.038) one      (0.023) the--    (0.018) mommy   
noun    0.989382
verb    0.010618
dtype: float32
gold = 'noun', pred = 'noun', correct = True
can you put them in hippo 's _____ ?
(0.028) mouth    | (0.227) foot     (0.124) bear     (0.061) coming   (0.050) hand     (0.039) food    
noun    0.873596
verb    0.126404
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you ____ to put those in the bin ?
(0.700) want     | (0.700) want     (0.238) going    (0.028) have     (0.016) need     (0.007) trying  
noun    0.000833
verb    0.999167
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to ___ those in the bin ?
(0.914) put      | (0.914) put      (0.021) get      (0.007) throw    (0.006) see      (0.006) take    
noun    0.00414
verb    0.99586
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to put those in the ___ ?
(0.197) bin      | (0.197) bin      (0.175) bag      (0.091) box    